In [ ]:
import instaloader
import openpyxl 
import datetime
import requests

In [ ]:
#今日の日付取得
today=str(datetime.date.today())

#Instagramのログイン情報
#Instagram ID
INSTAGRAM_ID = ''
#Instagramパスワード
INSTAGRAM_PASSWORD = ''
loader = instaloader.Instaloader()
loader.login(INSTAGRAM_ID, INSTAGRAM_PASSWORD)

#取りたい人のInstagram ID 今回は八人分の情報を取得しています
id =['','','','','','','','']
#エクセルファイルpath
excel_path=''
book = openpyxl.load_workbook(filename=excel_path)
sheet = book['シート1']
#フォロワー数リスト
artist_forower=[]
#前日のフォロワー数リスト
send_list=[]
#初日のフォロワー数
day1=[]
#スラックに送る内容
text_slack=[]
#エクセルシート番号
sheet_days=['A','C','E','G','I','K','M','O']
sheet_forower=['B','D','F','H','J','L','N','P']


In [ ]:
#アーティストフォロワー数取得
def insta_forowwer():
    i=0
    
    while True:
        profile = instaloader.Profile.from_username(loader.context, id[i])
        followers = profile.get_followers()
        forowescount = followers.count
        artist_forower.append(forowescount)
        i+=1
        if i==8:
            break

In [ ]:
#スラックに送る文章作成
def make_text():
  num=0
  num2=0
  num3=0
  max_sheet=str(sheet.max_row)
  #初日の数値取得
  while True:
      day1.append(sheet[sheet_forower[num]+str(3)].value)
      num+=1
      if num==8:
          break
  #前日の数値所得
  while True:
    send_list.append(sheet[sheet_forower[num2]+max_sheet].value)
    num2+=1
    if num2==8:
      break
    #スラックに送る文章作成
  while True:
    text_slack.append(id[num3]+'さん \r   '+str(artist_forower[num3])+'人  '+'増減数 '+str(int(artist_forower[num3])-int(sheet[sheet_forower[num3]+max_sheet].value)))
    num3+=1
    if num3==8:
      break

In [ ]:
def save_excel():
    #excelに書き込みを行う
    max_row = str(sheet.max_row+1)
    #日付の書き込み
    d=0
    f=0
    while True:
        sheet[sheet_days[d]+max_row]=today
        d+=1
        if d==8:
            break
    while True:
        sheet[sheet_forower[f]+max_row]=artist_forower[f]
        f+=1
        if f==8:
            break
    book.save(excel_path)

In [ ]:
def slack_send():
   #slack送信用
   #トークン記入
   TOKEN = ''
   #チャンネル名記入
   CHANNEL = 'フォロワー確認用'
　　#APIのURL
   url = ""
   send_message=0
   while True:
      headers = {"Authorization": "Bearer "+TOKEN}
      data  = {
         
      'channel': CHANNEL,
      #送りたいテキスト記入
      'text': text_slack[send_message]+'                    '
      }
      r = requests.post(url, headers=headers, data=data)
      print("return ", r.json())
      send_message+=1
      if send_message==8:
         break

In [ ]:
insta_forowwer()
make_text()
save_excel()
slack_send()